<div class="alert alert-block alert-info">Adapted from: https://github.com/salmanahmad4u/keras-iris/blob/master/iris_nn.py</div>

In [48]:
import csv
import numpy as np
import keras as kr
from keras.models import load_model

<div class="alert alert-block alert-info">Load the Iris dataset.
Data from: https://github.com/mwaskom/seaborn-data/blob/master/iris.csv</div>

In [49]:
iris = list(csv.reader(open('iris.csv')))[1:]

<div class="alert alert-block alert-info">The inputs are four floats: sepal length, sepal width, petal length, petal width.</div>

In [50]:
inputs  = np.array(iris)[:,:4].astype(np.float)

<div class="alert alert-block alert-info">Outputs are initially individual strings: setosa, versicolor or virginica.</div>

In [51]:
outputs = np.array(iris)[:,4]

Convert the output strings to ints.

In [52]:
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True)

# Encode the category integers as binary categorical vairables.

In [53]:
outputs_cats = kr.utils.to_categorical(outputs_ints)

# Split the input and output data sets into training and test subsets.

In [54]:
inds = np.random.permutation(len(inputs))
train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = inputs[test_inds],  outputs_cats[test_inds]

# Create a neural network.

In [55]:
model = kr.models.Sequential()

# Add an initial layer with 4 input nodes, and a hidden layer with 16 nodes.

In [56]:
model.add(kr.layers.Dense(16, input_shape=(4,)))

# Apply the sigmoid activation function to that layer.

In [57]:
model.add(kr.layers.Activation("sigmoid"))

# Add another layer, connected to the layer with 16 nodes, containing three output nodes.

In [58]:
model.add(kr.layers.Dense(3))

# Use the softmax activation function there.

In [59]:
model.add(kr.layers.Activation("softmax"))

# Configure the model for training.
# Uses the adam optimizer and categorical cross entropy as the loss function.
# Add in some extra metrics - accuracy being the only one.

In [60]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Fit the model using our training data.

In [61]:
model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
75/75 [==============================] - 0s - loss: 1.1042 - acc: 0.3600         
Epoch 2/100
75/75 [==============================] - 0s - loss: 1.0144 - acc: 0.4933        
Epoch 3/100
75/75 [==============================] - 0s - loss: 0.9594 - acc: 0.6933     
Epoch 4/100
75/75 [==============================] - 0s - loss: 0.9112 - acc: 0.6933     
Epoch 5/100
75/75 [==============================] - 0s - loss: 0.8630 - acc: 0.6933       
Epoch 6/100
75/75 [==============================] - 0s - loss: 0.8167 - acc: 0.6933       
Epoch 7/100
75/75 [==============================] - 0s - loss: 0.7757 - acc: 0.6933     
Epoch 8/100
75/75 [==============================] - 0s - loss: 0.7241 - acc: 0.6933       
Epoch 9/100
75/75 [==============================] - 0s - loss: 0.6815 - acc: 0.7200     
Epoch 10/100
75/75 [==============================] - 0s - loss: 0.6404 - acc: 0.7200     
Epoch 11/100
75/75 [==============================] - 0s - loss: 0.6124 - acc: 0.7200 

75/75 [==============================] - 0s - loss: 0.1389 - acc: 0.9733     
Epoch 91/100
75/75 [==============================] - 0s - loss: 0.1373 - acc: 0.9733     
Epoch 92/100
75/75 [==============================] - 0s - loss: 0.1345 - acc: 0.9867     
Epoch 93/100
75/75 [==============================] - 0s - loss: 0.1350 - acc: 0.9733     
Epoch 94/100
75/75 [==============================] - 0s - loss: 0.1337 - acc: 0.9867     
Epoch 95/100
75/75 [==============================] - 0s - loss: 0.1310 - acc: 0.9600     
Epoch 96/100
75/75 [==============================] - 0s - loss: 0.1283 - acc: 0.9867     
Epoch 97/100
75/75 [==============================] - 0s - loss: 0.1288 - acc: 0.9733     
Epoch 98/100
75/75 [==============================] - 0s - loss: 0.1287 - acc: 0.9600     
Epoch 99/100
75/75 [==============================] - 0s - loss: 0.1257 - acc: 0.9733     
Epoch 100/100
75/75 [==============================] - 0s - loss: 0.1301 - acc: 0.9867     


# Evaluate the model using the test data set.

In [62]:
loss, accuracy = model.evaluate(inputs_test, outputs_test, verbose=1)

32/75 [===========>..................] - ETA: 0s

# Output the accuracy of the model.

In [63]:
print("\n\nLoss: %6.4f\tAccuracy: %6.4f" % (loss, accuracy))



Loss: 0.1145	Accuracy: 0.9867


# Predict the class of a single flower.

In [64]:
prediction = np.around(model.predict(np.expand_dims(inputs_test[0], axis=0))).astype(np.int)[0]
print("Actual: %s\tEstimated: %s" % (outputs_test[0].astype(np.int), prediction))
print("That means it's a %s" % outputs_vals[prediction.astype(np.bool)][0])

Actual: [0 1 0]	Estimated: [0 1 0]
That means it's a versicolor


# Save the model to a file for later use.

In [65]:
model.save("iris_nn.h5")

# Load the model again with: model = load_model("iris_nn.h5")

In [66]:
model = load_model("iris_nn.h5")